# Timer

In [3]:
from time import sleep
from datetime import datetime as dt

start_time = "13.01.2023, 17:25"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [1]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

ttype = 'sell'
pattern = ['STOCH', 'RSI', 'Trend']
indicator_list = pattern
indicator_list_higher = ['Trend']

work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 2000
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Indicator_list'] = indicator_list
configs['Higher_TF_indicator_list'] = indicator_list_higher
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'RSI': {
                        'timeperiod': [14], 
                        'low_bound': [35],
                        'vol_window': [48], # [6, 12, 24, 48, 72, 96],
                        'vol_q_high': [0.25], # np.arange(0.1, 1, 0.05),
                        'vol_q_low': [0.9], # np.arange(0.1, 1, 0.05),
                        'first_candle': [0.55], # np.arange(0.5, 1, 0.05),
                        'second_candle': [0.75] # np.arange(0.5, 1, 0.05)
                       },
                'STOCH': {
                          'fastk_period': [9],
                          'slowk_period': [7],
                          'slowd_period': [3], 
                          'low_bound': [25]
                        },
                'Trend': {
                          'timeperiod':[48], 
                          'low_bound': [0]
                        }
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 15m/1h, trade type is sell
Number of combinations is 50


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [05:27<00:00,  6.55s/it]


# Check local statistics

In [2]:
# candles
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg')
stat.head(20)

,pattern,RSI_timeperiod,RSI_low_bound,RSI_vol_window,RSI_vol_q_high,RSI_vol_q_low,RSI_first_candle,RSI_second_candle,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
288,STOCH_RSI,14,35,48,0.90,0.10,0.55,0.75,9,7,3,25,9,1.811333,-2.904167,7.302000,-26.137500,0.8727,1.5576,2.1085,1.9055,1.9878,2.0245,2.1002,2.4794,2.3419,1.9266,1.9929,1.9200,1.9200,1.4488,1.4265,1.4265,1.4265,1.4265,1.4265,1.6171,1.7397,2.0379,2.1468,2.2116,-0.29,-0.58,-0.85,-1.11,-1.39,-1.67,-1.95,-2.24,-2.51,-2.77,-3.01,-3.24,-3.47,-3.62,-3.76,-3.87,-3.96,-4.03,-4.08,-4.14,-4.16,-4.23,-4.34,-4.43
270,STOCH_RSI,14,35,48,0.85,0.10,0.55,0.75,9,7,3,25,18,1.634758,-2.755833,11.425650,-49.605000,0.7682,1.1374,1.7567,1.6729,1.6089,1.5905,1.6464,1.8734,1.8284,1.7034,1.7226,1.7509,1.7479,1.4800,1.4600,1.4317,1.4317,1.4563,1.5281,1.6526,1.7240,1.9849,2.1198,2.1575,-0.28,-0.56,-0.82,-1.09,-1.36,-1.62,-1.87,-2.13,-2.38,-2.61,-2.84,-3.06,-3.28,-3.43,-3.57,-3.68,-3.77,-3.84,-3.89,-3.94,-3.94,-3.99,-4.06,-4.13
234,STOCH_RSI,14,35,48,0.75,0.10,0.55,0.75,9,7,3,25,39,1.348604,-2.695833,13.595562,-105.137500,0.9442,1.1357,1.6346,1.5071,1.4037,1.1929,1.2589,1.3856,1.3877,1.3818,1.4343,1.4902,1.5147,1.3074,1.3021,1.2958,1.2833,1.2961,1.2919,1.3189,1.3170,1.3909,1.4391,1.4526,-0.26,-0.52,-0.78,-1.03,-1.28,-1.53,-1.78,-2.02,-2.26,-2.50,-2.74,-2.97,-3.18,-3.36,-3.54,-3.67,-3.78,-3.87,-3.91,-3.93,-3.91,-3.92,-3.96,-4.00
289,STOCH_RSI,14,35,48,0.90,0.15,0.55,0.75,9,7,3,25,10,1.753604,-2.640833,7.536042,-26.408333,0.8097,1.4600,1.9764,1.6762,1.7572,1.8018,1.8868,2.1489,2.0429,1.7299,1.7678,1.6954,1.6954,1.5972,1.5689,1.5689,1.5689,1.5689,1.5689,1.7400,1.8500,2.1176,2.2153,2.2735,-0.27,-0.54,-0.79,-1.04,-1.29,-1.55,-1.81,-2.06,-2.30,-2.54,-2.75,-2.95,-3.16,-3.29,-3.42,-3.51,-3.59,-3.65,-3.69,-3.73,-3.74,-3.81,-3.91,-3.99
235,STOCH_RSI,14,35,48,0.75,0.15,0.55,0.75,9,7,3,25,40,1.352579,-2.635417,14.103167,-105.416667,0.9299,1.1247,1.6193,1.4747,1.3785,1.1786,1.2438,1.3618,1.3625,1.3563,1.4032,1.4543,1.4780,1.3477,1.3413,1.3347,1.3220,1.3344,1.3293,1.3551,1.3524,1.4240,1.4711,1.4843,-0.26,-0.52,-0.76,-1.01,-1.26,-1.51,-1.74,-1.98,-2.22,-2.45,-2.68,-2.90,-3.11,-3.29,-3.46,-3.59,-3.70,-3.78,-3.81,-3.83,-3.81,-3.82,-3.86,-3.90
252,STOCH_RSI,14,35,48,0.80,0.10,0.55,0.75,9,7,3,25,26,1.223988,-2.630833,5.823675,-68.401667,0.6192,0.8434,1.2752,1.2243,1.2272,0.9772,1.0680,1.2382,1.2424,1.2135,1.2200,1.2804,1.3059,1.2118,1.2073,1.1989,1.1837,1.1986,1.2422,1.3178,1.3554,1.5131,1.5946,1.6174,-0.25,-0.50,-0.75,-0.99,-1.24,-1.48,-1.72,-1.97,-2.20,-2.43,-2.66,-2.88,-3.10,-3.27,-3.44,-3.56,-3.67,-3.76,-3.81,-3.85,-3.84,-3.87,-3.92,-3.98
271,STOCH_RSI,14,35,48,0.85,0.15,0.55,0.75,9,7,3,25,19,1.625879,-2.626250,11.891704,-49.898750,0.7441,1.1120,1.7158,1.5841,1.5335,1.5221,1.5790,1.7699,1.7263,1.6155,1.6227,1.6415,1.6389,1.5682,1.5462,1.5181,1.5181,1.5411,1.6084,1.7252,1.7921,2.0367,2.1631,2.1985,-0.27,-0.54,-0.80,-1.05,-1.31,-1.56,-1.80,-2.05,-2.28,-2.50,-2.71,-2.92,-3.12,-3.26,-3.40,-3.50,-3.58,-3.65,-3.70,-3.74,-3.74,-3.78,-3.85,-3.92
306,STOCH_RSI,14,35,48,0.95,0.10,0.55,0.75,9,7,3,25,4,1.038158,-2.585000,0.152633,-10.340000,0.7158,1.1523,1.4400,1.2345,1.3444,1.4538,1.4908,1.5111,1.5270,0.9557,0.9383,0.8685,0.8685,0.5375,0.5476,0.5476,0.5476,0.5476,0.5476,0.8

In [2]:
# vol
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_rank')
stat.head(20)

,pattern,RSI_timeperiod,RSI_low_bound,RSI_vol_window,RSI_vol_q_high,RSI_vol_q_low,RSI_first_candle,RSI_second_candle,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
3,STOCH_RSI,14,35,48,0.25,0.9,0.55,0.75,9,7,3,25,181,1.075667,-2.006667,13.695667,-363.206667,0.8576,1.0390,1.2435,1.1634,1.0938,1.0830,1.1345,1.1733,1.1360,1.1295,1.1225,1.1148,1.0841,1.0579,1.0619,1.0437,1.0248,1.0014,1.0020,0.9971,1.0178,1.0726,1.0965,1.0653,-0.22,-0.43,-0.64,-0.85,-1.05,-1.25,-1.43,-1.62,-1.79,-1.97,-2.13,-2.29,-2.44,-2.56,-2.66,-2.73,-2.77,-2.80,-2.80,-2.79,-2.75,-2.73,-2.73,-2.73
4,STOCH_RSI,14,35,72,0.25,0.9,0.55,0.75,9,7,3,25,181,1.055633,-2.006667,10.069633,-363.206667,0.8543,1.0397,1.2244,1.1446,1.0771,1.0653,1.1048,1.1433,1.1084,1.1040,1.1004,1.0937,1.0642,1.0339,1.0385,1.0212,1.0033,0.9808,0.9819,0.9772,0.9982,1.0526,1.0765,1.0469,-0.22,-0.43,-0.64,-0.85,-1.05,-1.25,-1.43,-1.62,-1.79,-1.97,-2.13,-2.29,-2.44,-2.56,-2.66,-2.73,-2.77,-2.80,-2.80,-2.79,-2.75,-2.73,-2.73,-2.73
5,STOCH_RSI,14,35,96,0.25,0.9,0.55,0.75,9,7,3,25,180,1.065042,-2.013333,11.707500,-362.400000,0.8630,1.0476,1.2327,1.1513,1.0838,1.0717,1.1142,1.1535,1.1177,1.1127,1.1088,1.1017,1.0716,1.0466,1.0509,1.0330,1.0145,0.9914,0.9922,0.9873,1.0082,1.0631,1.0870,1.0565,-0.22,-0.43,-0.64,-0.85,-1.05,-1.25,-1.44,-1.62,-1.80,-1.97,-2.14,-2.30,-2.45,-2.57,-2.67,-2.74,-2.78,-2.81,-2.81,-2.80,-2.76,-2.74,-2.74,-2.74
2,STOCH_RSI,14,35,24,0.25,0.9,0.55,0.75,9,7,3,25,177,1.066933,-2.022500,11.847200,-357.982500,0.8747,1.0608,1.2613,1.1824,1.1117,1.0894,1.1271,1.1561,1.1194,1.1148,1.1118,1.1064,1.0765,1.0460,1.0505,1.0274,1.0007,0.9769,0.9781,0.9736,0.9953,1.0491,1.0736,1.0428,-0.22,-0.43,-0.65,-0.85,-1.06,-1.26,-1.45,-1.63,-1.81,-1.98,-2.15,-2.31,-2.46,-2.58,-2.68,-2.75,-2.79,-2.82,-2.83,-2.81,-2.77,-2.75,-2.75,-2.75
1,STOCH_RSI,14,35,12,0.25,0.9,0.55,0.75,9,7,3,25,172,1.066213,-2.030417,11.388550,-349.231667,0.8746,1.0741,1.2703,1.1923,1.1177,1.0929,1.1301,1.1566,1.1191,1.1118,1.1102,1.1065,1.0761,1.0506,1.0550,1.0312,1.0014,0.9751,0.9709,0.9615,0.9826,1.0368,1.0615,1.0302,-0.22,-0.43,-0.65,-0.85,-1.06,-1.26,-1.45,-1.63,-1.81,-1.99,-2.16,-2.32,-2.47,-2.60,-2.70,-2.77,-2.81,-2.84,-2.84,-2.82,-2.78,-2.76,-2.76,-2.75
0,STOCH_RSI,14,35,6,0.25,0.9,0.55,0.75,9,7,3,25,162,1.095225,-2.067083,15.426450,-334.867500,0.8426,1.0595,1.2085,1.1448,1.0814,1.0762,1.1334,1.1846,1.1521,1.1446,1.1495,1.1475,1.1278,1.0978,1.1050,1.0836,1.0675,1.0424,1.0367,1.0290,1.0497,1.1031,1.1287,1.0894,-0.22,-0.43,-0.65,-0.85,-1.06,-1.26,-1.46,-1.64,-1.83,-2.01,-2.18,-2.35,-2.50,-2.63,-2.74,-2.82,-2.87,-2.90,-2.91,-2.90,-2.86,-2.84,-2.85,-2.85


In [3]:
# vol window
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_rank')
stat.head(20)

,pattern,RSI_timeperiod,RSI_low_bound,RSI_vol_window,RSI_vol_q_high,RSI_vol_q_low,RSI_first_candle,RSI_second_candle,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
3,STOCH_RSI,14,35,48,0.25,0.9,0.55,0.75,9,7,3,25,181,1.075667,-2.006667,13.695667,-363.206667,0.8576,1.0390,1.2435,1.1634,1.0938,1.0830,1.1345,1.1733,1.1360,1.1295,1.1225,1.1148,1.0841,1.0579,1.0619,1.0437,1.0248,1.0014,1.0020,0.9971,1.0178,1.0726,1.0965,1.0653,-0.22,-0.43,-0.64,-0.85,-1.05,-1.25,-1.43,-1.62,-1.79,-1.97,-2.13,-2.29,-2.44,-2.56,-2.66,-2.73,-2.77,-2.80,-2.80,-2.79,-2.75,-2.73,-2.73,-2.73
4,STOCH_RSI,14,35,72,0.25,0.9,0.55,0.75,9,7,3,25,181,1.055633,-2.006667,10.069633,-363.206667,0.8543,1.0397,1.2244,1.1446,1.0771,1.0653,1.1048,1.1433,1.1084,1.1040,1.1004,1.0937,1.0642,1.0339,1.0385,1.0212,1.0033,0.9808,0.9819,0.9772,0.9982,1.0526,1.0765,1.0469,-0.22,-0.43,-0.64,-0.85,-1.05,-1.25,-1.43,-1.62,-1.79,-1.97,-2.13,-2.29,-2.44,-2.56,-2.66,-2.73,-2.77,-2.80,-2.80,-2.79,-2.75,-2.73,-2.73,-2.73
5,STOCH_RSI,14,35,96,0.25,0.9,0.55,0.75,9,7,3,25,180,1.065042,-2.013333,11.707500,-362.400000,0.8630,1.0476,1.2327,1.1513,1.0838,1.0717,1.1142,1.1535,1.1177,1.1127,1.1088,1.1017,1.0716,1.0466,1.0509,1.0330,1.0145,0.9914,0.9922,0.9873,1.0082,1.0631,1.0870,1.0565,-0.22,-0.43,-0.64,-0.85,-1.05,-1.25,-1.44,-1.62,-1.80,-1.97,-2.14,-2.30,-2.45,-2.57,-2.67,-2.74,-2.78,-2.81,-2.81,-2.80,-2.76,-2.74,-2.74,-2.74
2,STOCH_RSI,14,35,24,0.25,0.9,0.55,0.75,9,7,3,25,177,1.066933,-2.022500,11.847200,-357.982500,0.8747,1.0608,1.2613,1.1824,1.1117,1.0894,1.1271,1.1561,1.1194,1.1148,1.1118,1.1064,1.0765,1.0460,1.0505,1.0274,1.0007,0.9769,0.9781,0.9736,0.9953,1.0491,1.0736,1.0428,-0.22,-0.43,-0.65,-0.85,-1.06,-1.26,-1.45,-1.63,-1.81,-1.98,-2.15,-2.31,-2.46,-2.58,-2.68,-2.75,-2.79,-2.82,-2.83,-2.81,-2.77,-2.75,-2.75,-2.75
1,STOCH_RSI,14,35,12,0.25,0.9,0.55,0.75,9,7,3,25,172,1.066213,-2.030417,11.388550,-349.231667,0.8746,1.0741,1.2703,1.1923,1.1177,1.0929,1.1301,1.1566,1.1191,1.1118,1.1102,1.1065,1.0761,1.0506,1.0550,1.0312,1.0014,0.9751,0.9709,0.9615,0.9826,1.0368,1.0615,1.0302,-0.22,-0.43,-0.65,-0.85,-1.06,-1.26,-1.45,-1.63,-1.81,-1.99,-2.16,-2.32,-2.47,-2.60,-2.70,-2.77,-2.81,-2.84,-2.84,-2.82,-2.78,-2.76,-2.76,-2.75
0,STOCH_RSI,14,35,6,0.25,0.9,0.55,0.75,9,7,3,25,162,1.095225,-2.067083,15.426450,-334.867500,0.8426,1.0595,1.2085,1.1448,1.0814,1.0762,1.1334,1.1846,1.1521,1.1446,1.1495,1.1475,1.1278,1.0978,1.1050,1.0836,1.0675,1.0424,1.0367,1.0290,1.0497,1.1031,1.1287,1.0894,-0.22,-0.43,-0.65,-0.85,-1.06,-1.26,-1.46,-1.64,-1.83,-2.01,-2.18,-2.35,-2.50,-2.63,-2.74,-2.82,-2.87,-2.90,-2.91,-2.90,-2.86,-2.84,-2.85,-2.85


In [3]:
# trend
e_ratio_cols = [f'e_ratio_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]
pct_price_diff_cols = [f'pct_price_diff_{lag + 1}' for lag in range(24)]

stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['mar_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['mar_rank'] = stat['mar_avg'] * stat['forecasts_num']

stat = stat[[c for c in stat.columns if c not in e_ratio_cols + pct_price_diff_cols] + e_ratio_cols + 
            pct_price_diff_cols].sort_values('mar_avg')
stat.head(20)

,pattern,RSI_timeperiod,RSI_low_bound,RSI_vol_window,RSI_vol_q_high,RSI_vol_q_low,RSI_first_candle,RSI_second_candle,STOCH_fastk_period,STOCH_slowk_period,STOCH_slowd_period,STOCH_low_bound,Trend_timeperiod,Trend_low_bound,forecasts_num,e_ratio_avg,mar_avg,e_ratio_rank,mar_rank,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24
21,STOCH_RSI_Trend,14,35,48,0.25,0.9,0.55,0.75,9,7,3,25,44,0,132,1.324346,-2.137917,42.813650,-282.205000,1.0132,1.2267,1.4932,1.3942,1.3215,1.3329,1.4004,1.4579,1.4405,1.4467,1.4168,1.4190,1.3701,1.3487,1.3592,1.3223,1.2751,1.2369,1.2171,1.1988,1.2120,1.2696,1.2928,1.3187,-0.22,-0.44,-0.65,-0.86,-1.08,-1.29,-1.48,-1.68,-1.87,-2.06,-2.24,-2.41,-2.57,-2.72,-2.83,-2.92,-2.98,-3.02,-3.03,-3.03,-2.99,-2.97,-2.98,-2.99
18,STOCH_RSI_Trend,14,35,48,0.25,0.9,0.55,0.75,9,7,3,25,38,0,139,1.275792,-2.137917,38.335042,-297.170417,0.9958,1.1980,1.4345,1.3369,1.2540,1.2675,1.3394,1.3939,1.3774,1.3791,1.3550,1.3598,1.3156,1.3022,1.3134,1.2715,1.2311,1.1967,1.1810,1.1645,1.1795,1.2342,1.2563,1.2817,-0.22,-0.44,-0.65,-0.86,-1.08,-1.28,-1.48,-1.67,-1.86,-2.05,-2.24,-2.41,-2.57,-2.72,-2.84,-2.92,-2.98,-3.02,-3.04,-3.03,-2.99,-2.98,-2.98,-3.00
30,STOCH_RSI_Trend,14,35,48,0.25,0.9,0.55,0.75,9,7,3,25,62,0,119,1.372496,-2.134167,44.327004,-253.965833,1.0163,1.2504,1.5379,1.4379,1.3521,1.3547,1.4320,1.5102,1.4923,1.5099,1.4837,1.4805,1.4504,1.4281,1.4461,1.3981,1.3403,1.2946,1.2715,1.2500,1.2503,1.2974,1.3185,1.3367,-0.22,-0.43,-0.65,-0.86,-1.07,-1.27,-1.47,-1.67,-1.86,-2.05,-2.23,-2.41,-2.57,-2.71,-2.83,-2.91,-2.97,-3.02,-3.03,-3.03,-2.99,-2.98,-2.99,-3.00
31,STOCH_RSI_Trend,14,35,48,0.25,0.9,0.55,0.75,9,7,3,25,64,0,118,1.351104,-2.133333,41.430292,-251.733333,1.0048,1.2404,1.5214,1.4232,1.3389,1.3392,1.4148,1.4939,1.4765,1.4903,1.4582,1.4560,1.4254,1.4009,1.4157,1.3688,1.3123,1.2678,1.2456,1.2247,1.2259,1.2734,1.2948,1.3136,-0.22,-0.43,-0.65,-0.86,-1.07,-1.28,-1.48,-1.67,-1.86,-2.05,-2.24,-2.41,-2.57,-2.71,-2.83,-2.91,-2.97,-3.01,-3.03,-3.02,-2.99,-2.97,-2.98,-2.99
24,STOCH_RSI_Trend,14,35,48,0.25,0.9,0.55,0.75,9,7,3,25,50,0,125,1.314071,-2.127917,39.258854,-265.989583,0.9820,1.2120,1.4824,1.3813,1.2979,1.3017,1.3736,1.4391,1.4190,1.4319,1.4105,1.4118,1.3668,1.3539,1.3729,1.3312,1.2814,1.2411,1.2203,1.2012,1.2032,1.2524,1.2752,1.2949,-0.22,-0.43,-0.65,-0.86,-1.07,-1.28,-1.47,-1.66,-1.85,-2.04,-2.23,-2.40,-2.56,-2.70,-2.82,-2.90,-2.97,-3.01,-3.03,-3.02,-2.98,-2.97,-2.97,-2.98
27,STOCH_RSI_Trend,14,35,48,0.25,0.9,0.55,0.75,9,7,3,25,56,0,123,1.320329,-2.127917,39.400488,-261.733750,0.9681,1.2065,1.4849,1.3910,1.3042,1.3075,1.3808,1.4526,1.4349,1.4472,1.4267,1.4274,1.3806,1.3641,1.3817,1.3370,1.2846,1.2434,1.2230,1.2035,1.2054,1.2555,1.2787,1.2986,-0.22,-0.43,-0.65,-0.85,-1.07,-1.27,-1.47,-1.66,-1.85,-2.04,-2.23,-2.40,-2.56,-2.71,-2.82,-2.91,-2.97,-3.01,-3.03,-3.02,-2.98,-2.97,-2.97,-2.98
26,STOCH_RSI_Trend,14,35,48,0.25,0.9,0.55,0.75,9,7,3,25,54,0,123,1.320329,-2.127917,39.400488,-261.733750,0.9681,1.2065,1.4849,1.3910,1.3042,1.3075,1.3808,1.4526,1.4349,1.4472,1.4267,1.4274,1.3806,1.3641,1.3817,1.3370,1.2846,1.2434,1.2230,1.2035,1.2054,1.2555,1.2787,1.2986,-0.22,-0.43,-0.65,-0.85,-1.07,-1.27,-1.47,-1.66,-1.85,-2.04,-2.23,-2.40,-2.56,-2.71,-2.82,-2.91,-2.97,-3.01,-3.03,-3.02,-2.98,-2.97,-2.97,-2.98
25,STOCH_RSI_Trend,14,35,48,0.25,0.9,0.55,0.75,9,7,3,25,52,0,123,1.320329,-2.127917,39.400488,-261.733750,0.9681,1.20

# Research parameter influence

In [81]:
param = 'Trend_timeperiod'

res = stat.groupby([param]).agg({'e_ratio_avg': 'mean',
                                 'pct_price_diff_avg': 'mean',
                                 'e_ratio_rank': 'mean', 
                                 'price_rank': 'mean', 
                                 'forecasts_num': 'sum'}).sort_values(param).reset_index()
res

,Trend_timeperiod,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank,forecasts_num
0,2,1.937883,-0.562604,65.569898,-39.616042,247
1,3,2.747457,-1.018333,76.114432,-44.385000,123
2,4,4.568197,-2.099375,56.411997,-33.292188,91
3,5,3.548132,-1.371806,74.680826,-39.263194,80
4,6,3.757032,-1.289444,80.305944,-37.298889,80
5,8,3.517467,-1.098056,77.010065,-33.492500,86


# Plot result in file

In [77]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

x, y = 'Trend_timeperiod', 'pct_price_diff_avg'
ax.plot(res[x], res[y])
ax.set_xlabel(x)
ax.set_ylabel(y)
ax.set_title('_'.join(pattern) + '_' + ttype)

fig.savefig('temp.png', dpi=fig.dpi)

# Save new config data to config file

In [5]:
from config_updater import ConfigUpdater

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {
                'RSI': {
                        'timeperiod': [14], 
                        'low_bound': [35],
                        'vol_window': [48],
                        'vol_q_high': [0.25],
                        'vol_q_low': [0.9], 
                        'first_candle': [0.55], 
                        'second_candle': [0.75] 
                       },
                'STOCH': {
                          'fastk_period': [9],
                          'slowk_period': [7],
                          'slowd_period': [3], 
                          'low_bound': [25]
                        },
                'Trend': {
                          'timeperiod':[48], 
                          'low_bound': [0]
                        }
             }
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [3]:
import numpy as np
import pandas as pd
from glob import glob

ttype = 'sell'
pattern = ['STOCH', 'RSI']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['RSI_timeperiod', 
                           'RSI_low_bound', 
                           'STOCH_fastk_period', 
                           'STOCH_slowk_period', 
                           'STOCH_slowd_period', 
                           'STOCH_low_bound']).agg({'pct_right_forecast_avg': 'mean',
                                                    'pct_price_diff_avg': 'mean',
                                                    'forecast_rank': 'mean', 
                                                    'price_rank': 'mean', 
                                                    'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                         ascending=False)
total_stat.head(20)

pct_right_forecast_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                           
16             30            9                  7                  3                  30                            83.541667   
                                                                                      25                            85.785833   
12             25            9                  7                  3                  30                            88.782083   
14             30            9                  7                  3                  25                            80.833333   
12             25            9                  7                  3                  25                            87.847917   
14             25            9                  7                  3                  30                            89.395000   
                                                                                      20                            89.395000   
                                                                                      25                            89.395000   
12             25            9                  7                  3                  20                            89.395000   
16             30            9                  7                  3                  20                            84.820833   
14             30            9                  7                  3                  30                            78.804583   
16             25            9                  7                  3                  20                            88.750000   
                                                                                      25                            88.750000   
                                                                                      30                            88.750000   
12             30            9                  7                  3                  30                            75.416667   
14             30            9                  7                  3                  20                            79.687500   
16             15            5                  4                  3                  25                            93.229167   
12             25            9                  7                  3                  15                            91.664583   
14             25            9                  7                  3                  15                            91.664583   
16             15            7                  4                  5                  25                            95.025694   

                                                                                                       pct_price_diff_avg  \
RSI_timeperiod RSI_low_bound STOCH_fastk_period STOCH_slowk_period STOCH_slowd_period STOCH_low_bound                       
16             30            9                  7                  3                  30                        -1.760000   
                                                                                      25                        -1.970833   
12             25            9                  7                  3                  30                        -2.084167   
14             30            9                  7                  3                  25                        -1.777917   
12             25            9                  7                  3                  25                        -2.421250   
14             25            9                  7                  3                  30                        -2.759583   
                                                                                      20                        -2.759583   
                                                                                      25                        -2.759583   
12            